In [37]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
from lyricsgenius import Genius
import re

punct = '''!()-[]{};:"\,<>./?@#$%^&*_~'''

NameError: name 'lyricsgenius' is not defined

In [ ]:
bb_data = pd.read_csv("charts.csv")
bb_data['date'] = pd.to_datetime(bb_data.date, format='%Y-%m-%d')
# bb_data

## Get DF of individual year data
year = '2021'
s2021 = year+'-01-01'
e2021 = year+'-12-31'
mask = (bb_data['date'] > s2021) & (bb_data['date'] <= e2021)

# data_2021 = bb_data.loc[mask]
# data_2021

## Get dataframe grouped on year
grouped_data = bb_data.groupby(bb_data['date'].dt.year)
# for key, item in grouped_data:
    # print(grouped_data.get_group(key), "\n\n")

In [ ]:
genius = Genius(TOKEN)

## Get a single song's lyrics
song = genius.search_song("Hello", "Adele")
lyrics = song.lyrics

lyrics = re.sub(r'\[.*?\]', '', lyrics)
lyrics = lyrics.replace('\n', ' ')

for sym in lyrics:
  if sym in punct:
    lyrics = lyrics.replace(sym, "")
lyrics

Searching for "Hello" by Adele...
Done.


"Hello Lyrics Hello it's me I was wondering if after all these years you'd like to meet To go over everything They say that time's supposed to heal ya but I ain't done much healing Hello can you hear me I'm in California dreaming about who we used to be When we were younger and free I've forgotten how it felt before the world fell at our feet   There's such a difference between us And a million miles   Hello from the other side I must've called a thousand times To tell you I'm sorry for everything that I've done But when I call you never seem to be home Hello from the outside At least I can say that I've tried To tell you I'm sorry for breaking your heart But it don't matter it clearly doesn't tear you apart anymore  Hello how are you It's so typical of me to talk about myself I'm sorry I hope that you're well Did you ever make it out of that town where nothing ever happened   It's no secret that the both of us Are running out of time   So hello from the other side Other side I must've

In [ ]:
## Get all the songs' lyrics
for key in grouped_data.groups.keys():
    kdf = grouped_data.get(key)
    for row in kdf.iterrows():
        song = genius.search_song(row['song'], row['artist'])
        lyrics = song.lyrics

        lyrics = re.sub(r'\[.*?\]', '', lyrics)
        lyrics = lyrics.replace('\n', ' ')

        for sym in lyrics:
            if sym in punct:
                lyrics = lyrics.replace(sym, "")
            lyrics